In [30]:
import pandas as pd
pd.set_option('mode.chained_assignment', None) # suppress unnecessary warnings
import numpy as np
from os import listdir
import re

import sqlalchemy as sa
import cx_Oracle

from pandas.tseries.offsets import Day, MonthEnd
from dateutil.relativedelta import relativedelta
import datetime
import os

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from chart_studio.plotly import plot, iplot
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

In [138]:
co2 = pd.read_excel('owid-co2-data.xlsx')
co2 = co2.loc[(co2['year']<=2018)]
co2['decade'] = (co2['year'].astype(str).str.slice(0,3) + "0").astype(int)

In [139]:
co2.head()

,iso_code,country,year,co2,consumption_co2,co2_growth_prct,co2_growth_abs,trade_co2,co2_per_capita,consumption_co2_per_capita,...,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp,decade
0,AFG,Afghanistan,1949,0.015,NaN,NaN,NaN,NaN,0.002,NaN,...,NaN,NaN,NaN,NaN,7624058.0,NaN,NaN,NaN,NaN,1940
1,AFG,Afghanistan,1950,0.084,NaN,475.0,0.070,NaN,0.011,NaN,...,NaN,NaN,NaN,NaN,7752117.0,9.421400e+09,NaN,NaN,NaN,1950
2,AFG,Afghanistan,1951,0.092,NaN,8.7,0.007,NaN,0.012,NaN,...,NaN,NaN,NaN,NaN,7840151.0,9.692280e+09,NaN,NaN,NaN,1950
3,AFG,Afghanistan,1952,0.092,NaN,0.0,0.000,NaN,0.012,NaN,...,NaN,NaN,NaN,NaN,7935996.0,1.001732e+10,NaN,NaN,NaN,1950
4,AFG,Afghanistan,1953,0.106,NaN,16.0,0.015,NaN,0.013,NaN,...,NaN,NaN,NaN,NaN,8039684.0,1.063052e+10,NaN,NaN,NaN,1950


In [140]:
co2.columns

Index(['iso_code', 'country', 'year', 'co2', 'consumption_co2',
       'co2_growth_prct', 'co2_growth_abs', 'trade_co2', 'co2_per_capita',
       'consumption_co2_per_capita', 'share_global_co2', 'cumulative_co2',
       'share_global_cumulative_co2', 'co2_per_gdp', 'consumption_co2_per_gdp',
       'co2_per_unit_energy', 'coal_co2', 'cement_co2', 'flaring_co2',
       'gas_co2', 'oil_co2', 'other_industry_co2', 'cement_co2_per_capita',
       'coal_co2_per_capita', 'flaring_co2_per_capita', 'gas_co2_per_capita',
       'oil_co2_per_capita', 'other_co2_per_capita', 'trade_co2_share',
       'share_global_cement_co2', 'share_global_coal_co2',
       'share_global_flaring_co2', 'share_global_gas_co2',
       'share_global_oil_co2', 'share_global_other_co2',
       'cumulative_cement_co2', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_oil_co2',
       'cumulative_other_co2', 'share_global_cumulative_cement_co2',
       'share_global_cumulative_c

In [141]:
corr = co2.dropna(subset=['iso_code']).groupby('country').agg({'cumulative_co2':'max','gdp':'mean'}).dropna()

In [142]:
world_gdp = co2.loc[(co2['country']=='World')&(co2['year']==co2['year'].max())][['country','gdp']].iloc[0]['gdp']
world_gdp

113630168331870.0

In [169]:
co2_cumu = co2.loc[(co2['country']!='World')&(co2['year']==co2['year'].max())].dropna(subset=['iso_code'])
co2_cumu = co2_cumu[['country','cumulative_co2','share_global_cumulative_co2','co2_per_capita','gdp','cumulative_cement_co2', 'cumulative_coal_co2', 
                     'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_oil_co2',  'cumulative_other_co2', 'share_global_cumulative_cement_co2', 
                     'share_global_cumulative_coal_co2', 'share_global_cumulative_flaring_co2',  'share_global_cumulative_gas_co2', 
                     'share_global_cumulative_oil_co2', 'share_global_cumulative_other_co2']]
co2_cumu['share_global_cumulative_co2_rank'] = co2_cumu['share_global_cumulative_co2'].rank(ascending=False)
co2_cumu['gdp_rank'] = co2_cumu['gdp'].fillna(0).rank(ascending=False)
co2_cumu['share_of_global_gdp'] = co2_cumu['gdp'].div(world_gdp)
co2_cumu['gdp_in_trillion'] = co2_cumu['gdp'].div(1e12)
co2_cumu.head(2)

,country,cumulative_co2,share_global_cumulative_co2,co2_per_capita,gdp,cumulative_cement_co2,cumulative_coal_co2,cumulative_flaring_co2,cumulative_gas_co2,cumulative_oil_co2,...,share_global_cumulative_cement_co2,share_global_cumulative_coal_co2,share_global_cumulative_flaring_co2,share_global_cumulative_gas_co2,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,share_global_cumulative_co2_rank,gdp_rank,share_of_global_gdp,gdp_in_trillion
69,Afghanistan,170.655,0.01,0.281,6.759497e+10,2.399,48.425,NaN,19.533,94.341,...,0.01,0.01,NaN,0.01,0.02,NaN,129.0,99.0,0.000595,0.067595
292,Albania,276.916,0.02,1.874,3.400911e+10,17.924,66.130,NaN,16.144,176.718,...,0.04,0.01,NaN,0.01,0.03,NaN,109.0,122.0,0.000299,0.034009


In [172]:
top = 15
co2_top = co2_cumu.sort_values(by='share_global_cumulative_co2',ascending=False).head(top)
co2_top.loc[co2_top['gdp_rank']>top]

,country,cumulative_co2,share_global_cumulative_co2,co2_per_capita,gdp,cumulative_cement_co2,cumulative_coal_co2,cumulative_flaring_co2,cumulative_gas_co2,cumulative_oil_co2,...,share_global_cumulative_cement_co2,share_global_cumulative_coal_co2,share_global_cumulative_flaring_co2,share_global_cumulative_gas_co2,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,share_global_cumulative_co2_rank,gdp_rank,share_of_global_gdp,gdp_in_trillion
22972,Ukraine,29326.134,1.81,5.236,4.313199e+11,404.595,16529.530,159.483,6120.321,6005.938,...,1.01,2.18,0.96,2.66,1.06,3.08,10.0,42.0,0.003796,0.431320
17848,Poland,27238.724,1.69,8.905,1.053479e+12,400.374,22677.871,101.996,1111.660,2821.002,...,1.00,2.99,0.61,0.48,0.50,3.65,11.0,24.0,0.009271,1.053479
20240,South Africa,20243.681,1.25,8.167,6.732721e+11,238.055,17565.417,NaN,175.873,2264.335,...,0.59,2.32,NaN,0.08,0.40,NaN,13.0,32.0,0.005925,0.673272
1579,Australia,17770.925,1.10,16.706,1.238560e+12,185.504,9874.001,254.109,1910.549,5444.741,...,0.46,1.30,1.53,0.83,0.96,2.96,15.0,19.0,0.010900,1.238560


In [160]:
print("There are {} nations in the dataset for the year {}.".format(co2_cumu.country.count(),co2['year'].max()))
print("Top {} carbon emitters covered {:.2f}% of the global carbon emission.".format(top,co2_top.share_global_cumulative_co2.sum()))

There are 214 nations in the dataset for the year 2018.
Top 15 carbon emitters covered 75.29% of the global carbon emission.


In [161]:
after = co2.loc[(co2['country']=='World')&(co2['year']>=1991)]
print("There are {:.0f} billion tonnes of CO2 emitted from {} through {}.".format(after.co2.sum()/1000,after.year.min(),after.year.max()))

There are 813 billion tonnes of CO2 emitted from 1991 through 2018.


In [162]:
before = co2.loc[(co2['country']=='World')&(co2['year']<1991)]
print("There are {:.0f} billion tonnes of CO2 emitted from {} through {}.".format(before.co2.sum()/1000,before.year.min(),before.year.max()))

There are 804 billion tonnes of CO2 emitted from 1750 through 1990.


In [176]:
#Exporting
#co2_top.to_excel('co2_top.xlsx',index=False)

In [ ]:
df = co2.loc[co2['country']!='World'].dropna(subset=['iso_code']).sort_values(by='co2',ascending=False)
fig = px.scatter(df, x="co2", y="gdp",color='country')
fig.show()

In [ ]:
df = co2[['year','co2','consumption_co2','coal_co2', 'cement_co2', 'flaring_co2','gas_co2', 'oil_co2', 'other_industry_co2']].groupby('year').sum().reset_index()
fig = px.bar(df, x="year", y=['co2','consumption_co2'], title="Consumption vs Production", barmode='group')
fig.show()